In [10]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [11]:
files = filenames= os.listdir ("FT-data-DSpace/")
files

['EdixiXMLExport_20091.xml',
 'EdixiXMLExport_20101.xml',
 'EdixiXMLExport_20102.xml',
 'EdixiXMLExport_20111.xml',
 'EdixiXMLExport_20121.xml',
 'EdixiXMLExport_20131.xml',
 'EdixiXMLExport_20141.xml',
 'EdixiXMLExport_20142.xml',
 'EdixiXMLExport_20151.xml',
 'EdixiXMLExport_20161.xml']

In [12]:
data = []
for file in files:
    print(file)
    tree = ET.parse('FT-data-DSpace/'+file)
    root = tree.getroot()
    for leaf in root:
        if leaf.findall('Møde'): ## if we have meetings in the leaf node
            for meeting in leaf:
                if meeting.tag == 'Møde':
                    meeting_id = meeting.find('MeetingId').text
                    meeting_date = meeting.find('DateOfSitting').text
                    meeting_number = meeting.find('Mødenummer').text
                    for agenda in meeting:
                        if agenda.tag == 'Dagsordenpunkt':
                        ### extract basic information about the meeting
                            try:
                                agenda_number = agenda.find('Punktnummer').text
                            except:
                                agenda_number = ''
                            try:
                                names = agenda.find('Mødetitel').findall('Exitus')
                                agenda_name = []
                                for name in names:
                                    agenda_name.append(name[0][0].text)
                            ## missing PreText
                            except Exception as e:
                                agenda_name = ''
                            #print(e)
                            try:
                                file_type = agenda.find('Sagstype').text
                            except:
                                file_type = ''
                            try:
                                file_number = agenda.find('Sagsnummer').text
                            except:
                                file_number = ''
                            try:
                                file_step = agenda.find('Sagstrin').text
                            except:
                                file_step = ''
                        ### extract speakers from the meeting
                            for speech in agenda.findall('Tale'):
                                try:
                                    person = speech.find('Navn').text
                                except:
                                    person = ''
                                try:
                                    role = speech.find('Rolle').text
                                except:
                                    role = ''
                                try:
                                    start = speech.find('Starttid').text
                                except:
                                    start = ''
                                try:
                                    end = speech.find('Sluttid').text
                                except:
                                    end = ''
                                try:
                                    text = speech.find('Tekst').text
                                except:
                                    text = ''
                            #print(person, role, start, end, text)
                                data.append([meeting_id, meeting_date, meeting_number, agenda_name, agenda_number,
                                      file_type, file_number, file_step, person, role, start, end, text])
                        
                        
                    
df = pd.DataFrame(data,  columns=['ID', 'Date', 'Meeting Number', 'Agenda Name', 'Agenda Number', 'File Type', 
                          'File Number', 'File Step', 'Speaker Name', 'Speaker Role', 'Speaker Start',
                          'Speaker End', 'Speech'])
df.sample(3)

EdixiXMLExport_20091.xml
EdixiXMLExport_20101.xml
EdixiXMLExport_20102.xml
EdixiXMLExport_20111.xml
EdixiXMLExport_20121.xml
EdixiXMLExport_20131.xml
EdixiXMLExport_20141.xml
EdixiXMLExport_20142.xml
EdixiXMLExport_20151.xml
EdixiXMLExport_20161.xml


,ID,Date,Meeting Number,Agenda Name,Agenda Number,File Type,File Number,File Step,Speaker Name,Speaker Role,Speaker Start,Speaker End,Speech
188150,2185F3AA-706E-49A3-BDAC-CE5DB9907107,2012-11-29T10:00:00,28. møde,"[13) Forespørgsel nr. F 8:, Forespørgsel til t...",13,Forespørgsel,8,BEH1,Bertel Haarder,formand,2012-11-29T13:49:04.110,2012-11-29T13:49:05.017,Ane Halsboe-Larsen.
53019,786A67F8-C34F-4AA2-A0D9-DC9B192AE80E,2010-02-03T13:00:00,48. møde,"[Spm. nr. S 1002, 35) Til forsvarsministeren a...",35,Spørgsmål til spørgetid,1002,,Søren Gade,minister,2010-02-03T18:51:58.520,2010-02-03T18:53:05.673,"Jeg kan bekræfte, at jeg har et tæt forhold ti..."
334689,DEFF77C6-C51F-4DCD-9CEA-081D1CE42EA7,2017-02-08T13:00:00,57. møde,"[Spm. nr. S 710, 6) Til udenrigsministeren af:...",6,Spørgsmål til spørgetid,710,,None,formand,2017-02-08T13:42:07.337,2017-02-08T13:42:15.097,Tak for det. Spørgsmålet er sluttet.Det næste ...


In [13]:
df.to_pickle('FT-data-DSpace/data_processed.pickle')

In [40]:
df[['Speaker Name', 'Speech']]

,Speaker Name,Speech
0,None,Mødet er åbnet.I dag er der følgende anmeldels...
1,Mogens Lykketoft,Der er ikke mere at foretage i dette møde.Folk...
2,None,"Forhandlingen er åbnet, og den første, der får..."
3,Peter Juel Jensen,Jeg holder ordførertalen for Venstre og på veg...
4,None,Tak til hr. Peter Juel Jensen. Der er ikke øns...
...,...,...
61557,Bente Dahl,Vi diskuterer igen og igen farligheden af de h...
61558,Helge Adam Møller,Tak til ordførerne. Så er det ordføreren for f...
61559,Per Clausen,"Jeg skal starte med at takke for en, synes jeg..."
61560,Per Clausen,"Det andet, som jeg også synes er vigtigt, og s..."
